## 🔖목차
- **1. GPT 3.5 turbo 모델에 Fine-tuning 수행**
  - Fine-tuning 데이터셋 불러오기
  - Fine-tuning용 jsonl 파일 생성
  - Fine-tuning 데이터셋을 API 환경에 업로드
  - Fine-tuning Job 생성
  - Fine-tuning 작업 상태 확인
- **2. Fine-tuned 모델로 뉴스레터 생성**
  - 예시 데이터셋에 적용
  - 실제 뉴스 데이터셋에 적용

---

## **1. GPT 3.5 turbo 모델에 Fine-tuning 수행**
- 일관된 형태로 양질의 뉴스레터를 생성하기 위해 해당 작업에 특화된 모델로 미세 조정함

In [ ]:
# 필요한 라이브러리 초기화
import pandas as pd
import json
import openai

In [ ]:
# GPT API 클라이언트 초기화
api_key = "본인의 API key를 입력"
openai.api_key = api_key

### 1.1. Fine-tuning 데이터셋 불러오기

In [ ]:
# Tuning에 사용할 데이터 40건 불러오기
data = pd.read_csv('tuning_data_ver3.csv', encoding = 'utf-8-sig')

In [ ]:
data

In [ ]:
# Input 데이터 예시 확인
print(data.iloc[0, 0])

In [ ]:
# Output 데이터 예시 확인
print(data.iloc[0, 3])

### 1.2. Fine-tuning용 jsonl 파일 생성

In [ ]:
# 튜닝 데이터를 위한 빈 리스트를 생성
tuning_data = []

for _, row in data.iterrows():
    example = {
        "messages": [
           {
                "role": "system",
                "content": """당신은 초등학생들을 위해 원본 신문기사를 뉴스레터 형태로 바꿔서 작성해주는 기자입니다. 어려운 신문 기사를 뉴스레터 형태로 바꿔서 작성한다면, 학생들이 본문 내용을 더욱 쉽게 이해할 수 있을 것입니다. 아래의 조건들을 바탕으로, [원본 신문기사]를 뉴스레터 형태로 작성해주세요. 조건 1 : 모든 문장에서 학생들에게 친밀감 있는 말투를 사용해주세요. 조건 2 : 적절한 이모지를 사용해주세요. 조건 3 : 결과물은 4개의 Part으로 구성해주세요. 구성은 다음과 같습니다. 1st Part : 간단한 인사와 함께 [원본 신문기사]의 주제에 대해서 소개해주세요. 2nd Part : [원본 신문기사]에 등장하는 단어들 중, [어려운 어휘]에 속하는 단어들을 쉽게 풀어서 설명해주세요. 3rd Part : [원본 신문기사]의 본문을 초등학생이 이해하기 쉬운 말로 풀어서 작성해주세요. 단, 본문의 내용이 지나치게 요약되면 안 됩니다. 본문 내용을 최대한 보존한 상태에서, 어려운 표현들만 쉽게 풀어서 작성해주세요. 4th Part : 본문 내용을 정리하면서 마무리 인사를 해주세요. 조건 4 : 글의 전반적인 구성과 형태를 뉴스레터처럼 구성해주세요."""
            },
            {
                "role": "user",
                "content": '[원본 신문기사] : {}'.format(row["Input"])
            },
            {
                "role": "user",
                "content": '[어려운 어휘] : {}'.format(row["어려운 어휘"])
            },
            {
                "role": "assistant",
                "content": row["수정본"]
            }
        ]
    }
    tuning_data.append(example)

In [ ]:
tuning_data

In [ ]:
# json formatted data to jsonl → fine_tuning_data2 → fine_tuning_data3로 이름 바꿈
with open("fine_tuning_data3.jsonl" , encoding= "utf-8", mode="w") as file: 
    for i in tuning_data: 
        file.write(json.dumps(i) + "\n")

In [ ]:
# 확인(한 줄씩 빈 행이 나오는 건 정상, 원 데이터에는 빈 행 없음)
with open("fine_tuning_data3.jsonl") as f: 
    for line in f: print(line)

In [ ]:
len(tuning_data)

### 1.3. Fine-tuning 데이터셋을 API 환경에 업로드
- 앞서 만든 "fine_tuning_data3.jsonl" 파일

In [ ]:
# Set OpenAI API keys
openai.api_key = api_key

# File names is 'processed_data.jsonL
with open('fine_tuning_data3.jsonl', 'rb') as file:
    response = openai.File.create(
        file = file,
        purpose = 'fine-tune')

In [ ]:
# 업로드 된 파일 ID 확인
file_id = response['id']
print(f'File uploaded successfully with ID: {file_id}')

### 1.4. Fine-tuning Job 생성

In [ ]:
# Send fine-tuning request
tuning_job = openai.FineTuningJob.create(
    training_file = file_id,
    model = 'gpt-3.5-turbo'
    )

In [ ]:
# 모델 job ID 확인
fine_tuning_job_id = tuning_job['id']
print(f"Fine-tuning job created successfully with ID : {fine_tuning_job_id}")

### 1.5. Fine-tuning 작업 상태 확인

In [ ]:
# 작업 상태 확인
job = openai.FineTuningJob.retrieve(id=fine_tuning_job_id)

# 작업 상태 출력 (시간은 좀 걸릴 수도 있음, succeeded가 나오면 완료)
print(job['status'])

In [ ]:
# Fine-tuning 완료 후 모델 ID 확인
fine_tune_status = openai.FineTuningJob.retrieve(fine_tuning_job_id)
model_id = fine_tune_status["fine_tuned_model"]

print(f"Created Fine-tuned model with ID : {model_id}")

## **2. Fine-tuning 모델 사용**

### 2.1. 예시 데이터셋에 적용
- 실제 뉴스 본문 데이터셋에 적용한 내용은 아래 2.2절 참고

In [ ]:
# GPT API 클라이언트 초기화
openai.api_key = "⭐본인의 API key를 입력"
# Fine-tuned 모델 ID를 입력
model_id = '⭐Fine-tuning 완료한 모델의 ID를 입력'

In [ ]:
# 뉴스레터로 변환할 기사 원본 불러오기 (여기서는 예시 데이터 사용)
ex_data = pd.read_csv('ex_data.csv', encoding = 'utf-8-sig')

In [ ]:
ex_data

In [ ]:
article = ex_data.iloc[0, 0] # '원본 신문 기사'가 담긴 컬럼
to_change_words = ex_data.iloc[0, 1] # '설명해야 할 어려운 단어'가 담긴 컬럼

In [ ]:
# 기본 프롬프트 설정
prompt = [ 
    {
        "role": "system",
        "content": """당신은 초등학생들을 위해 원본 신문기사를 뉴스레터 형태로 바꿔서 작성해주는 기자입니다. 어려운 신문 기사를 뉴스레터 형태로 바꿔서 작성한다면, 학생들이 본문 내용을 더욱 쉽게 이해할 수 있을 것입니다. 아래의 조건들을 바탕으로, [원본 신문기사]를 뉴스레터 형태로 작성해주세요. 조건 1 : 모든 문장에서 학생들에게 친밀감 있는 말투를 사용해주세요. 조건 2 : 적절한 이모지를 사용해주세요. 조건 3 : 결과물은 4개의 Part으로 구성해주세요. 구성은 다음과 같습니다. 1st Part : 간단한 인사와 함께 [원본 신문기사]의 주제에 대해서 소개해주세요. 2nd Part : [원본 신문기사]에 등장하는 단어들 중, [어려운 어휘]에 속하는 단어들을 쉽게 풀어서 설명해주세요. 3rd Part : [원본 신문기사]의 본문을 초등학생이 이해하기 쉬운 말로 풀어서 작성해주세요. 단, 본문의 내용이 지나치게 요약되면 안 됩니다. 본문 내용을 최대한 보존한 상태에서, 어려운 표현들만 쉽게 풀어서 작성해주세요. 4th Part : 본문 내용을 정리하면서 마무리 인사를 해주세요. 조건 4 : 글의 전반적인 구성과 형태를 뉴스레터처럼 구성해주세요."""
    },
    {
        "role": "user",
        "content": '[원본 신문기사] : {}'.format(article)
    },
    {
        "role": "user",
        "content": '[어려운 어휘] : {}'.format(to_change_words)
    }
]


In [ ]:
prompt

In [ ]:
# Fine-tuned GPT 모델에 메시지 보내기
response = openai.ChatCompletion.create(
    model=model_id,
    messages=prompt
)

In [ ]:
# 결과 확인
chat_response = response['choices'][0]['message']['content'].strip()

In [ ]:
# Fine-tuned GPT의 답변 결과 출력
print(chat_response)

### 2.2. 실제 뉴스 데이터셋에 적용
- 뉴스레터 생성 절차
    1. 신문 기사를 입력 받으면 자동으로 어려운 어휘를 산출함
    2. 어려운 어휘와 함께 신문 기사를 파인튜닝된 모델에 입력
    3. 어린이 맞춤형 뉴스레터로 출력 (총 2,237건 생성 완료)

In [ ]:
# ------------------------------------- # 0. 어려운 어휘 산출을 위한 calculate_noun_difficulty 함수 준비

# 바른 형태소 분석기 설정
import bareunpy
from bareunpy import Tagger
API_KEY="⭐본인의 API key를 입력"
my_tagger = Tagger(API_KEY, 'localhost')

# 기타 라이브러리 불러오기
import os
import sys
import json
import time
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from collections import Counter

# 빈도 수 데이터 불러오기
corp_freq = pd.read_csv('~~일상빈도 데이터~~.csv', encoding = 'utf-8-sig') # 일상 빈도 데이터
corp_freq.drop(['Unnamed: 0', 'ratio(%)'], inplace = True, axis = 1)
mean = np.mean(corp_freq['frequency']) # 평균 이하 형태소는 잘라내기
daily_usage = corp_freq[corp_freq['frequency'] >= mean]

wave_corp = pd.read_csv('~~물결21 빈도 데이터~~.csv', encoding='utf-8-sig') # 물결 21 빈도 데이터
wave_corp.drop(['Unnamed: 0.1', 'Unnamed: 0'], inplace = True, axis = 1)

# 불용어 정의
stopwords = ['요구', '특파원', '참석자', '희생자', '기자', '지난해', '양국', '갑', '을', '중대형', '승용차', '이듬해', '핸드볼',
             '국가', '연합뉴스', '당국', '지난해', '기업', '상승세', '닷새', '누리집', '꼴찌', '사망자', '이날', '대통령', '지역',
             '시인', '메시지', '센터', '시', '의료원', '붕괴', '기자실', '보고서', '소폭', '라이벌', '노조', '내년도', '견제', '앵커',
             '논설위원', '수락', '리서치', '타임스', '무죄', '뉴시스', '도', '조사', '상당수', '지난달', '마다', '주', '가운데', '개방',
            '연평균', '고속버스', '평균', '관계자', '고교', '연면적', '참가자', '당시', '주석', '선수단']

# 각 기사 본문의 NF-iDF를 계산하는 함수 정의
def calculate_noun_difficulty(input_data, daily_usage, wave_corp, my_tagger, desired_pos_tags=['NNG'], stopwords=None):
    start_time = time.time()
    score_column = []

    for i, line in enumerate(input_data['본문'], start=1):
        score_list = []
        res = my_tagger.tags([line])

        total_usage = 0
        filtered_result = [(word, pos) for word, pos in res.pos() if pos in desired_pos_tags]
        tokens = [word for word, _ in filtered_result]

        for token in tokens:
            daily_freq = daily_usage.loc[daily_usage['corpus'] == token, 'frequency'].mean()
            wave_freq = wave_corp.loc[wave_corp['단어'] == token, '총 빈도수'].mean()

            if np.isnan(daily_freq):
                score = wave_freq / 1
            else:
                score = wave_freq / daily_freq

            score_tuple = (token, score)
            score_list.append(score_tuple)

        score_column.append(score_list)

        if i % 10 == 0:
            elapsed_time = time.time() - start_time
            print(f"Score Calculating Processed {i} samples. Elapsed time: {elapsed_time:.2f} seconds")

    input_data['점수_tuple'] = score_column

    total_word_list = []
    for i in range(len(input_data)):
        line = input_data.loc[i, '점수_tuple']
        word_list = [word for (word, _) in line]
        total_word_list.append(word_list)

    input_data['counter'] = total_word_list

    total_result_list = []
    for l in input_data['counter']:
        asd = Counter(l)
        word_frequency_pairs = list(asd.items())
        result_list = []
        result_list.extend(word_frequency_pairs)
        total_result_list.append(result_list)

    input_data['counter'] = total_result_list

    result_list = []

    for i in range(len(input_data)):
        list1 = input_data.loc[i, '점수_tuple']
        list2 = input_data.loc[i, 'counter']

        new_tuple_list = []

        for tup1 in list1:
            word = tup1[0]
            matching_tup2 = next((tup for tup in list2 if tup[0] == word), None)

            if matching_tup2 is not None:
                new_tuple = (word, matching_tup2[1] * tup1[1])
                new_tuple_list.append(new_tuple)

        result_list.append(new_tuple_list)

    input_data['단어난이도'] = result_list

    final_list = []

    for i in range(len(input_data)):
        line = input_data.loc[i, '단어난이도']
        line_without_nan = [(word, num) for word, num in line if not pd.isna(num)]
        sorted_result = sorted(line_without_nan, key=lambda x: x[1], reverse=True)

        unique_values = set()
        unique_result = [(word, num) for word, num in sorted_result if word not in unique_values and not unique_values.add(word)]

        if stopwords:
            unique_result = [(word, num) for word, num in unique_result if word not in stopwords]

        final_list.append(unique_result)

    input_data['단어난이도'] = final_list

    num_list = []

    for i in range(len(input_data)):
        line = input_data.loc[i, '단어난이도']
        for (_, num) in line:
            num_list.append(num)

    top6_list = []

    for i in range(len(input_data)):
        line = input_data.loc[i, '단어난이도']
        lbyl_list = []
        for (word, num) in line:
            lbyl_list.append(word)
            if len(lbyl_list) == 6:
                break
        top6_list.append(lbyl_list)

    input_data['top6'] = top6_list #👈GPT에 전달할 어려운 어휘 Top6

    final_result = []

    for i in range(len(input_data)):
        list1 = input_data.loc[i, '단어난이도']
        단어난이도_list = [tup[1] for tup in list1]

        단어난이도_sum = sum(단어난이도_list) / len(단어난이도_list)
        final_result.append(단어난이도_sum)

    input_data['명사난이도 평균 점수'] = final_result #👈기사별 NF-iDF 측정

    total_time = time.time() - start_time
    print(f"Final Score Calculated elapsed time: {total_time:.2f} seconds")
    
    return input_data

In [ ]:
# ------------------------------------- # 1. 실제 데이터셋에 대해 어려운 어휘 산출

# 뉴스레터로 변환할 신문 기사 데이터 로드
input_data = pd.read_csv('~~신문기사 원본 데이터셋~~.csv', encoding = 'utf-8-sig')

# calculate_noun_difficulty 함수를 사용해 '어려운 어휘 Top6'를 산출
result_data = calculate_noun_difficulty(input_data=input_data, 
                                        daily_usage=daily_usage, 
                                        wave_corp=wave_corp, 
                                        my_tagger=my_tagger, 
                                        stopwords=stopwords)

In [ ]:
# ------------------------------------- # 2. 신문기사 & 어려운 어휘를 Fine-tuned GPT에 전달

# 전체 데이터에 대한 gpt_answer 열 추가
result_data['gpt_answer'] = ''

start_time = time.time()
for index, row in result_data.iterrows(): 
    article = row['본문'] #👈Fine-tuned GPT에게 입력할 '기사 본문'
    to_change_words = row['top6'] #👈Fine-tuned GPT에게 입력할 '어려운 어휘'
    
    to_change_again = []
    for word in to_change_words :
        to_change_again.append(word)
    
    to_change_again = ', '.join(to_change_again)

    # prompt 정의
    prompt = [
        {
            "role": "system",
            "content": """당신은 초등학생들을 위해 원본 신문기사를 뉴스레터 형태로 바꿔서 작성해주는 기자입니다. 어려운 신문 기사를 뉴스레터 형태로 바꿔서 작성한다면, 학생들이 본문 내용을 더욱 쉽게 이해할 수 있을 것입니다. 아래의 조건들을 바탕으로, [원본 신문기사]를 뉴스레터 형태로 작성해주세요. 조건 1 : 모든 문장에서 학생들에게 친밀감 있는 말투를 사용해주세요. 조건 2 : 적절한 이모지를 사용해주세요. 조건 3 : 결과물은 4개의 Part으로 구성해주세요. 구성은 다음과 같습니다. 1st Part : 간단한 인사와 함께 [원본 신문기사]의 주제에 대해서 소개해주세요. 2nd Part : [원본 신문기사]에 등장하는 단어들 중, [어려운 어휘]에 속하는 단어들을 쉽게 풀어서 설명해주세요. 3rd Part : [원본 신문기사]의 본문을 초등학생이 이해하기 쉬운 말로 풀어서 작성해주세요. 단, 본문의 내용이 지나치게 요약되면 안 됩니다. 본문 내용을 최대한 보존한 상태에서, 어려운 표현들만 쉽게 풀어서 작성해주세요. 4th Part : 본문 내용을 정리하면서 마무리 인사를 해주세요. 조건 4 : 글의 전반적인 구성과 형태를 뉴스레터처럼 구성해주세요."""
        },
        {
            "role": "user",
            "content": '[원본 신문기사] : {}'.format(article)
        },
        {
            "role": "user",
            "content": '[어려운 어휘] : {}'.format(to_change_again)
        }
    ]

    # Fine-tuned GPT에 쿼리 보내기
    chat_completion = client.chat.completions.create(
        messages=prompt,
        model=model_id,
    )

    # Fine-tuned GPT의 답변을 받아서 데이터프레임에 추가
    result_data.at[index, 'gpt_answer'] = chat_completion.choices[0].message.content
    
    # 샘플 10개당 시간 측정
    if index % 1 == 0:
        elapsed_time = time.time() - start_time
        print(f"Processed {index}th samples. Elapsed time: {elapsed_time:.2f} seconds")

In [ ]:
# ------------------------------------- # 3. 어린이 맞춤형 뉴스레터 2,237건 출력 완료 

result_data.head() # 결과 확인
result_data.to_csv('~~뉴스레터 생성 완료 데이터셋~~.csv', encoding = 'utf-8-sig', index = False) # CSV 파일로 저장